# Question 1

![](FIN30270_Asst3_Pt1.png)

In [8]:
using Random, Statistics, Ipopt, JuMP,LinearAlgebra, Distributions

In [2]:
nsim = 1000;
w = -100 * log.(rand(Float64, (nsim)));

modeli = Model(Ipopt.Optimizer)
set_silent(modeli)
@variable(modeli, 0 <= x)
@variable(modeli, 0 <= u[1:nsim])
for i in 1:nsim
    @constraint(modeli, w[i] - x <= u[i])
end
@objective(modeli, Max, 1.5 * (mean(w) - mean(u)) - x)
optimize!(modeli)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In [3]:
x = value(x)

40.5485383368778

## Part b

In [4]:
model = Model(Ipopt.Optimizer)
set_silent(model)
@variable(model, 0 <= x)
@objective(model, Max, 1.5 * (100 * (1 - exp(-x/100))) - x) # lambda = 100
optimize!(model)
xAnalytical = value(x)

40.546510816935076

## Part c

In [5]:
nsim = 1000;
w = -50 * log.(rand(Float64, (nsim)));

model2 = Model(Ipopt.Optimizer)
set_silent(model2)
@variable(model2, 0 <= x)
@variable(model2, 0 <= u[1:nsim])
for i in 1:nsim
    @constraint(model2, w[i] - x <= u[i])
end
@objective(model2, Max, 1.5 * (mean(w) - mean(u)) - x)
optimize!(model2)
x = 2 * value(x)

43.332853908647714

## Part d

In [6]:
model = Model(Ipopt.Optimizer)
set_silent(model)
@variable(model, 0 <= x)
@objective(model, Max, (1.5 * (50 * (1 - exp(-x/50))) - x)) # lambda = 50
optimize!(model)
xAnalytical = 2 * value(x) # Multiply by 2 as we have 2 half days

40.54651082316822

## Part e

# Question 2

In [59]:
mu_2m = 0.08 * 2/12
sigma_2m = 0.2 * sqrt(2/12)
nsim = 1000
mu_ln = log(mu_2m^2/sqrt(mu_2m^2 + sigma_2m^2))
sigma_ln = log(1 + mu_2m^2/sigma_2m^2)
lnDist = rand(LogNormal(mu_ln, sigma_ln), (nsim))
# rets = (1 + mu_2m).^(rand((-2, 0, 0, 2), (nsim)))  .* exp.(rand(Normal(mu_2m, sigma_2m^2), (nsim))) # uu, ud, du, dd
# sqrt(var(rets)) * sqrt(12/2)


rets = exp.((0.08 - 0.2^2/2) * 2/12 .+ 0.2 * rand(Normal(0, 2/12), (nsim))) .* exp.((0.08 - 0.2^2/2) .* 2/12 .+ 0.2 * rand(Normal(0, 2/12), (nsim))) .* exp.(rand(Normal(mu_2m, sigma_2m^2), (nsim)));

In [60]:
model2 = Model(Ipopt.Optimizer)
set_silent(model2)
@variable(model2, 0 <= w[1:2] <= 1)
@constraint(model2, sum(w) == 1)
@constraint(model2, w[1] * sqrt(var([x for x in rets if x < 1.04])) * sqrt(6) <= 0.1)
@objective(model2, Max, w[1] * mean(rets))
optimize!(model2)

weights = value.(w)

2-element Vector{Float64}:
  1.0000000051612918
 -5.161291693932853e-9